## Main file project


In [1]:
# Main imports
import networkx as nx
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold 
from sklearn.metrics import f1_score
import json
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import MultinomialNB
import os 
import pandas as pd 
from sklearn.naive_bayes import MultinomialNB

In [2]:
# Load the training dataset (with the labels)
X = []
y = []
with open("./../Data/training.txt", "r") as f:
    for line in tqdm(f):
        line = line.split()
        X.append(np.array([int(line[0]), int(line[1])]))
        y.append(np.array(int(line[2])))
    X = np.array(X)
    y = np.array(y)
    

453797it [00:02, 168181.57it/s]
100%|██████████████████████████████████████████████████████████████████████████| 33226/33226 [00:12<00:00, 2757.02it/s]


0.0


In [ ]:
Node_info = [] 

i =0
j=0
for root, dirs, files in os.walk("./../Data/node_information/text", topdown=False):
        for name in tqdm(files):
            path = os.path.join("./../Data/node_information/text",name)
            try:
                f = open(path, "r",  encoding='utf-8', errors='ignore')
                Node_info.append(f.read())
            except:
                print(path)
                i+=1
            j+=1

print(i/j)

In [ ]:
Node_info = [] 

for nd in tqdm(X):
        f = open("./../Data/node_information/text/"+str(nd[0])+".txt", "r",  encoding='utf-8', errors='ignore')
        a= f.read()
        f = open("./../Data/node_information/text/"+str(nd[1])+".txt", "r",  encoding='utf-8', errors='ignore')
        a=a+f.read()
        Node_info.append(a)
            

 31%|██████████████████████▏                                                 | 140139/453797 [22:33<1:05:44, 79.52it/s]

## Define some functions

In [3]:
# Fill graph from np.arrays
def fill_graph(X, y):
    G = nx.Graph()
    for nd, v in tqdm(zip(X, y)):
        if int(v) == 1:
            G.add_edge(nd[0], nd[1])
    return G

def export_to_json(X,y):
    data = {}
    data["edges"]=[]
    data["nodes"]=[]
    i=0
    for _ in tqdm(X):
        data["nodes"].append({
            'id': str(_[0]),
            'label' : str(_[0]),
            'group' : 1
        })
        data["nodes"].append({
            'id': str(_[1]),
            'label' : str(_[1]),
            'group' : 1
        })
        i+=1
    
    for nd, v in tqdm(zip(X, y)):
        if int(v) == 1:
            data['edges'].append({
                'from': str(nd[0]),
                'to' : str(nd[1])
            })
            
    with open('./Vis/data.json', 'w') as outfile:
        json.dump(data, outfile)
    

# Compute jaccard predictions given a Graph and the nodes array
def jaccard_prediction(G, X):
    predictions = []
    for x in X:
        try:
            coef = [[u, v, p] for u, v, p in nx.jaccard_coefficient(G, [(x[0], x[1])])][0]
        except KeyError:  # If the node tryed isn't in the Graph we predict 0...
            coef = [0, 0, 0]
        if coef[2] > 0.005:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions


# Compute the score given a predictor and the training set

def computeTrainingScore_kFold(predictor, X, y, n_splits):
    kf = KFold(n_splits = n_splits)  # Define the split - into n_splits folds

    mean_score = 0
    i=1
    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        # Let's reinitilize the graph
        G = fill_graph(X_train, y_train)
        y_pred = predictor(G, X_val)
        mean_score += compute_score(y_pred, y_val)
        print(mean_score/i)
        i+=1
    return mean_score/n_splits

def computeTrainingScore(predictor, X, y, test_size=0.1):
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=test_size, random_state=1)
    G = fill_graph(X_train, y_train)
    y_pred = predictor(G, X_val)
    return compute_score(y_pred, y_val)

## Graph Visualization

In [ ]:
#G= fill_graph(X,y)
export_to_json(X,y)

## Only NLP, supervised learning

### Preparing data (bag of words + TFIDF transform)

In [4]:
count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

X_train_counts = count_vect.fit_transform(Node_info)
nodes_tfidf = tfidf_transformer.fit_transform(X_train_counts)


In [ ]:
print(nodes_tfidf.shape)

### Testing cossine similarity

In [ ]:
av1=0
av2=0
i,j=0,0
d=0
for nd, v in tqdm(zip(X, y)):
    if int(v) ==1:
        av1+=np.dot(np.transpose(nodes_tfidf[:,nd[0]]),nodes_tfidf[:,nd[1]])
        i+=1
    else:
        av2+=np.dot(np.transpose(nodes_tfidf[:,nd[0]]),nodes_tfidf[:,nd[1]])
        j+=1
    d+=1
    if(d==100): break;
print("Cos for linked edges : " + (str(av1/i)))
print("Cos for linked edges : " + (str(av2/j)))

### Naive Bayes Classificator

In [52]:
X_t = np.concatenate((Node_info[X[:,0][0]],Node_info[X[:,1][0]]), axis=0)
print(X_t)

ValueError: zero-dimensional arrays cannot be concatenated

In [34]:
X_train, X_test, y_train, y_test = train_test_split(df1, y, test_size=0.10, random_state=42)

clf = MultinomialNB().fit(X_train, y_train)
y_pred=clf.predict(X_test)
print(f1_score(y_test,y_pred))


ValueError: setting an array element with a sequence.

# Node2vec embedding 

## 1) Given approach - Graph baseline - Jaccard coefficient

In [ ]:
# For training purpose, as the testing.txt set doesn't have the labels
# We will split our original (training) dataset with kfolds 
# For each split we'll initilize a Graph, train our model and get the mean score
# On the training set
n_splits = 2

jacc_score = computeTrainingScore(jaccard_prediction, X, y)
print(jacc_score)